<h2>Lab4: Об’єднання потоків в Spark Streaming</h2>

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, explode, col, desc
from pyspark.sql.types import StringType, StructType, StructField, IntegerType, DoubleType    
import os

os.environ['SPARK_HOME'] = "/home/zaranik/.sdkman/candidates/spark/current"
os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
os.environ['PYSPARK_PYTHON'] = 'python3'

Створення Spark сессії

In [ ]:
spark = SparkSession.builder.appName("RunningCountExample").getOrCreate()
spark

Опис схеми даних

In [ ]:
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("entity", StringType(), True),
    StructField("code", StringType(), True),
    StructField("year", IntegerType(), True),
    StructField("human_food", DoubleType(), True),
    StructField("animal_feed", DoubleType(), True),
    StructField("processed", DoubleType(), True),
    StructField("timestamp", StringType(), True)
])

Створення статичного датасету

In [ ]:
static_data = [
  (1, "Americas","AMCS", 1961, 7000, 96000, 11656000), 
  (2, "Argentina", "ARG", 2004, 0, 0, 24229000), 
  (3, "Asia", "ASIA", 1984, 5954000, 520000, 11831000)
]
columns = ["id", "entity", "code", "year", "human_food", "animal_feed", "processed"]
static_df = spark.createDataFrame(static_data, columns)
static_df.show()

Створення спарк стріма, який читає усі .json файли вказаної директорії

In [ ]:
stream_df = spark.readStream.schema(schema).json("./")

Джоін статичного датафрейму та датафрейму стріма, що читає з файлу.
Джоін відбувається за полем id. Тобто, у результуючий датафрейм попадуть лише ті записи stream_df, id яких є у static_df

In [ ]:
merged_df = stream_df.join(static_df, on='id', how='inner')

Обираємо поля, які хочемо бачити у фінальному стрімі

In [ ]:
# Select columns for the final result
result = merged_df.select(
    stream_df['id'],
    stream_df['entity'],
    stream_df['code'],
    stream_df['year'],
    stream_df['human_food'],
    static_df['animal_feed'],
    static_df['processed']
)

Записуємо дані стріма у папку output_task1

In [ ]:
# Save the result to JSON
query = result.writeStream.outputMode("append") \
    .format("json") \
    .option("path", "./output_task1/") \
    .option("checkpointLocation", "./output_task1/checkpoints/") \
    .start()

In [ ]:
query.awaitTermination()

In [ ]:
spark.stop()